In [1]:
from d2l import torch as d2l
import torch
import torchvision
from torch import nn

https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html
nn.ReLU(alpha) 不带inplace

In [2]:
class G_block(nn.Module):
        def __init__(self, channels, nc=3, kernel_size=4, strides=2,
                 padding=1, alpha=0.2, **kwargs):
            super(G_block, self).__init__(**kwargs)
            self.conv2d = nn.ConvTranspose2d(
                nc, channels, kernel_size, strides, padding, bias=False)
            self.batch_norm = nn.BatchNorm2d(channels)
            self.activation = nn.ReLU(alpha)

        def forward(self, X):
            return self.activation(self.batch_norm(self.conv2d(X)))

In [3]:
x = torch.zeros((2, 3, 16, 16))
g_blk = G_block(20)
g_blk(x).shape

torch.Size([2, 20, 32, 32])

In [4]:
x = torch.zeros((2, 3, 1, 1))
g_blk = G_block(20, strides=1, padding=0)
g_blk(x).shape

torch.Size([2, 20, 4, 4])

In [5]:
def Conv2DTranspose(channels, kernel_size, use_bias, nc=3):
    return nn.ConvTranspose2d(nc, channels, kernel_size=kernel_size, bias=use_bias)

https://pytorch.org/docs/master/generated/torch.nn.Tanh.html

In [6]:
n_G = 64
net_G = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, n_G*8),  # Output: (64 * 4, 8, 8)
    G_block(n_G*2, n_G*4),  # Output: (64 * 2, 16, 16)
    G_block(n_G, n_G*2),   # Output: (64, 32, 32)
    Conv2DTranspose(
              3, kernel_size=4, strides=2, padding=1, use_bias=False),
    nn.Tanh()               # Output: (3, 64, 64)
)


TypeError: Conv2DTranspose() got an unexpected keyword argument 'strides'

In [7]:
def Conv2DTranspose(channels, kernel_size, strides, use_bias, nc=3):
    return nn.ConvTranspose2d(nc, channels, kernel_size=kernel_size,stride=strides bias=use_bias)

SyntaxError: invalid syntax (<ipython-input-7-54cf9b7c16e0>, line 2)

In [8]:
def Conv2DTranspose(channels, kernel_size, strides, use_bias, nc=3):
    return nn.ConvTranspose2d(nc, channels, kernel_size=kernel_size,stride=strides,bias=use_bias)

In [9]:
n_G = 64
net_G = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, n_G*8),  # Output: (64 * 4, 8, 8)
    G_block(n_G*2, n_G*4),  # Output: (64 * 2, 16, 16)
    G_block(n_G, n_G*2),    # Output: (64, 32, 32)
    Conv2DTranspose(
              3, kernel_size=4, strides=2, padding=1, use_bias=False),
    nn.Tanh()               # Output: (3, 64, 64)
)

TypeError: Conv2DTranspose() got an unexpected keyword argument 'padding'

In [10]:
def Conv2DTranspose(channels, kernel_size, strides, padding, use_bias, nc=3):
    return nn.ConvTranspose2d(nc, channels, kernel_size=kernel_size,stride=strides, padding=padding, bias=use_bias)

In [11]:
n_G = 64
net_G = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, n_G*8),  # Output: (64 * 4, 8, 8)
    G_block(n_G*2, n_G*4),  # Output: (64 * 2, 16, 16)
    G_block(n_G, n_G*2),    # Output: (64, 32, 32)
    Conv2DTranspose(
              3, nc=n_G, kernel_size=4, strides=2, padding=1, use_bias=False),
    nn.Tanh()               # Output: (3, 64, 64)
)

In [12]:
x = torch.zeros((1, 100, 1, 1))
net_G(x).shape

RuntimeError: Given transposed=1, weight of size [3, 512, 4, 4], expected input[1, 100, 1, 1] to have 3 channels, but got 100 channels instead

In [13]:
n_G = 64
net_G1 = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, n_G*8))

In [14]:
x = torch.zeros((1, 100, 1, 1))
net_G1(x).shape

RuntimeError: Given transposed=1, weight of size [3, 512, 4, 4], expected input[1, 100, 1, 1] to have 3 channels, but got 100 channels instead

In [16]:
n_G = 64
net_G1 = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4))

In [17]:
x = torch.zeros((1, 100, 1, 1))
net_G1(x).shape

RuntimeError: Given transposed=1, weight of size [3, 512, 4, 4], expected input[1, 100, 1, 1] to have 3 channels, but got 100 channels instead

In [18]:
n_G = 64
net_G1 = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, nc=n_G*8))

In [19]:
x = torch.zeros((1, 100, 1, 1))
net_G1(x).shape

RuntimeError: Given transposed=1, weight of size [3, 512, 4, 4], expected input[1, 100, 1, 1] to have 3 channels, but got 100 channels instead

https://discuss.pytorch.org/t/runtimeerror-given-groups-1-weight-of-size-10-512-1-1-expected-input-16-3-224-224-to-have-512-channels-but-got-3-channels-instead/56583

In [20]:
x = torch.zeros((1, 3, 1, 1))
net_G1(x).shape

torch.Size([1, 256, 8, 8])

In [21]:
n_G = 64
net_G = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, n_G*8),  # Output: (64 * 4, 8, 8)
    G_block(n_G*2, n_G*4),  # Output: (64 * 2, 16, 16)
    G_block(n_G, n_G*2),    # Output: (64, 32, 32)
    Conv2DTranspose(
              3, nc=n_G, kernel_size=4, strides=2, padding=1, use_bias=False),
    nn.Tanh()               # Output: (3, 64, 64)
)

In [22]:
x = torch.zeros((1, 3, 1, 1))
net_G(x).shape

torch.Size([1, 3, 64, 64])

In [23]:
x = torch.zeros((1, 100, 1, 1))
net_G.initialize()
net_G(x).shape

AttributeError: 'Sequential' object has no attribute 'initialize'

In [24]:
x = torch.zeros((1, 100, 1, 1))
net_G.apply()
net_G(x).shape

TypeError: apply() missing 1 required positional argument: 'fn'

In [25]:
x = torch.zeros((1, 100, 1, 1))
net_G.apply(x)
net_G(x).shape

TypeError: 'Tensor' object is not callable

https://cs230.stanford.edu/blog/pytorch/

https://codeleading.com/article/61792960264/

In [ ]:
x = torch.zeros((1, 100, 1, 1))


https://github.com/eriklindernoren/PyTorch-YOLOv3/issues/414#issuecomment-618931177

netD.zero_grad()
https://atcold.github.io/pytorch-Deep-Learning/en/week09/09-3/